In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import optimize

class bond_pricer:
    
    def __init__(self, 
                 principal_value, 
                 coupon_rate, 
                 settlement_date, 
                 maturity_date, 
                 coupon_frequency=1,
                 convention='30/360',
                 yield_to_maturity=None, 
                 price_dirty=None,
                 price_clean=None):
        
        self.principal_value = principal_value
        self.coupon_rate = coupon_rate
        self.coupon_frequency = coupon_frequency
        self.coupon_amount = self.principal_value * (self.coupon_rate / self.coupon_frequency)
        self.settlement_date = datetime.strptime(settlement_date, '%d/%m/%Y').date()
        self.maturity_date = datetime.strptime(maturity_date, '%d/%m/%Y').date()
        self.convention = convention
        self.n_months_coupon = int(12 / self.coupon_frequency)  
        
        # Compute cash flow dates
        cf_date = self.maturity_date
        cf_dates = [self.maturity_date]
        while cf_date + relativedelta(months=-self.n_months_coupon) > self.settlement_date:
            cf_date += relativedelta(months=-self.n_months_coupon)
            cf_dates.append(cf_date)        
        self.cash_flow_dates = cf_dates[::-1]
        
        # Compute accrued interest
        self.next_coupon_date = self.cash_flow_dates[0]
        self.prev_coupon_date = self.next_coupon_date + relativedelta(months=-self.n_months_coupon)
        if self.convention == 'Actual/Actual':
            self.n_days_accrued = (self.settlement_date - self.prev_coupon_date).days    
            self.n_days_curr_coupon = (self.next_coupon_date - self.prev_coupon_date).days
            self.accrual_period = self.n_days_accrued / self.n_days_curr_coupon
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == '30/360':
            d1 = min(30, self.prev_coupon_date.day)
            if d1 == 30:
                d2 = min(d1, self.settlement_date.day)
            else:
                d2 = self.settlement_date.day
            self.n_days_accrued = 360 * (self.settlement_date.year - self.prev_coupon_date.year) + 30 * (self.settlement_date.month - self.prev_coupon_date.month) + d2 - d1
            self.n_days_curr_coupon = 360 / self.coupon_frequency
            self.accrual_period = self.n_days_accrued / self.n_days_curr_coupon
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        
        # Compute cash flow amounts and time to cash flows and maturity
        first_tau = (self.n_days_curr_coupon - self.n_days_accrued) / self.n_days_curr_coupon        
        cf_amounts = []
        taus = []
        for i in range(len(self.cash_flow_dates)):
            cf_amounts.append(self.coupon_amount)
            taus.append((first_tau + i) / self.coupon_frequency)        
        cf_amounts[-1] += self.principal_value
        self.cash_flows = np.array(cf_amounts)
        self.taus = np.array(taus)
        self.time_to_maturity = self.taus[-1]
        
        # Compute clean and dirty prices
        if price_dirty:
            self.price_dirty = price_dirty
            self.price_clean = self.price_dirty - self.accrued_interest
        elif price_clean:
            self.price_clean = price_clean
            self.price_dirty = self.price_clean + self.accrued_interest
        elif yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency*np.log(1 + self.yield_to_maturity / self.coupon_frequency)
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
            self.price_dirty = np.sum(self.cash_flows * self.discount_factors)
            self.price_clean = self.price_dirty - self.accrued_interest
        
        # Compute current yield
        self.current_yield = (self.coupon_rate * self.principal_value) / self.price_dirty
        
        # Compute yield to maturity
        if yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency * np.log(1 + self.yield_to_maturity / self.coupon_frequency)
        else:
            def get_bond_price(yield_to_maturity, cash_flows, taus):
                return np.sum(cash_flows * np.exp(-yield_to_maturity * taus))
            
            get_ytm = lambda ytm: get_bond_price(ytm, self.cash_flows, self.taus) - self.price_dirty
            
            self.ytm_adjusted = optimize.newton(get_ytm, 0.04)
            self.yield_to_maturity = (np.exp(self.ytm_adjusted / self.coupon_frequency) - 1) * self.coupon_frequency
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
       
        # Compute duration
        self.first_derivative = np.sum(-self.taus * self.cash_flows * self.discount_factors)
        self.duration_macaulay = -1/self.price_dirty * self.first_derivative
        self.duration_modified = self.duration_macaulay / (1 + self.yield_to_maturity / self.coupon_frequency)                                                        
        
        # Compute convexity
        self.second_derivative = np.sum(self.taus**2 * self.cash_flows * self.discount_factors)
        self.convexity = 1/self.price_dirty * self.second_derivative
        
        # Generate summary table
        index = ['Dirty price','Clean price', 'Accrued interest', 'Face value', 'Coupon rate', 'Coupon frequency', 'Yield to maturity', 
                 'Time to maturity', 'Macaulay duration', 'Modified duration', 'Convexity', 'Settlement date', 'Maturity date', 'Convention']
        data = list(np.round([self.price_dirty, self.price_clean, self.accrued_interest, self.principal_value, self.coupon_rate, self.coupon_frequency, 
                self.yield_to_maturity, self.time_to_maturity, self.duration_macaulay, self.duration_modified, self.convexity],2))
        data += [self.settlement_date, self.maturity_date, self.convention]
        self.summary = pd.DataFrame(data, index, columns=[''])
        
    # Approximate new price after delta_ytm change in yield to maturity
    def approx_new_price(self, delta_ytm):
        return self.price_dirty * (1 + (-self.duration_modified * delta_ytm + 0.5 * self.convexity * delta_ytm**2))
    
    # Summary of the bond metrics
    def print_summary(self):
        print(self.summary)

In [2]:
# Parameters
principal_value = 100
coupon_rate = 0.0684
coupon_frequency = 12
settlement_date = '24/01/2019'
maturity_date = '19/04/2027'
yield_to_maturity = 0.05
convention='30/360'

# Instantiate bond
bond = bond_pricer(principal_value=principal_value, coupon_rate=coupon_rate, coupon_frequency=coupon_frequency, 
                settlement_date=settlement_date, maturity_date=maturity_date, 
                yield_to_maturity=yield_to_maturity, convention=convention)

# Print bond metrics
bond.print_summary()

                             
Dirty price             112.5
Clean price             112.4
Accrued interest          0.1
Face value              100.0
Coupon rate              0.07
Coupon frequency         12.0
Yield to maturity        0.05
Time to maturity         8.24
Macaulay duration        6.44
Modified duration        6.42
Convexity               48.44
Settlement date    2019-01-24
Maturity date      2027-04-19
Convention             30/360
